# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
from skimage.transform import resize

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [9]:
train_doc = np.random.permutation(open('C:/Users/Anindam Som/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('C:/Users/Anindam Som/Project_data/val.csv').readlines())
batch_size = 16

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = range(15)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(folder_list)/batch_size)
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,25,160,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate over the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    image = resize(image, (160, 120))
                    
                    batch_data[folder,idx,:,:,0] = (image[:,:,0] - image[:,:,0].min())/(image[:,:,0].max() - image[:,:,0].min())
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - image[:,:,1].min())/(image[:,:,1].max() - image[:,:,1].min())
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - image[:,:,2].min())/(image[:,:,2].max() - image[:,:,2].min())
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        if((folder_list.shape[0]%batch_size) == 0):
            remaining_batch = int(folder_list.shape[0] - (num_batches * batch_size))
            batch_data = np.zeros((batch_size,25,160,120,3))
            batch_labels = np.zeros((remaining_batch,5))
            for folder in range(1,remaining_batch+1):
                img = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+img[item]).astype(np.float32)
                    
                    image = resize(image, (160, 120))
                    
                    batch_data[folder,idx,:,:,0] = (image[:,:,0] - image[:,:,0].min())/(image[:,:,0].max() - image[:,:,0].min())
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - image[:,:,1].min())/(image[:,:,1].max() - image[:,:,1].min())
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - image[:,:,2].min())/(image[:,:,2].max() - image[:,:,2].min())
                    
                    batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
                    
            yield batch_data, batch_labels       
                

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [8]:
curr_dt_time = datetime.datetime.now()
train_path = 'C:/Users/Anindam Som/Project_data/train'
val_path = 'C:/Users/Anindam Som/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 60
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 60


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [7]:
# Model--1
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

model = Sequential()

input_shape = (25,160,120,3)

# 1st Layer
model.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu',
                 input_shape=input_shape))

# Maxpooling Layer--1
model.add(MaxPooling3D(pool_size=(3, 3, 3)))

# Adding Dropout--1
model.add(Dropout(0.5))

# 2nd Layer
model.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))

# Maxpooling Layer--2
model.add(MaxPooling3D(pool_size=(3, 3, 3)))

# Adding Flatten layers
model.add(Flatten())

# Adding Dropout--2
model.add(Dropout(0.5))

# Adding fully connected layer
model.add(Dense(128,kernel_initializer="normal",activation='relu')) 

# Adding Dropout--3
model.add(Dropout(0.5))

# softmax layer
model.add(Dense(5, kernel_initializer="normal",activation='softmax'))

# model summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_3 (Conv3D)            (None, 23, 158, 118, 32)  2624      
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 7, 52, 39, 32)     0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 7, 52, 39, 32)     0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 5, 50, 37, 64)     55360     
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 1, 16, 12, 64)     0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 12288)             0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 12288)             0         
__________

In [12]:
# Model--2 -- Using Batch Normalization

model1 = Sequential()
input_shape = (25,160,120,3)

# 1st Layer
model1.add(Conv3D(16, padding='same', kernel_size=(3, 3, 3),
                 activation='relu',
                 input_shape=input_shape))
model1.add(BatchNormalization())

# 2nd Layer
model1.add(Conv3D(16, padding='same', kernel_size=(3, 3, 3),
                 activation='relu',
                 input_shape=input_shape))
model1.add(BatchNormalization())

# Maxpooling Layer --1
model1.add(MaxPooling3D(pool_size=(2, 2, 2)))

# 3rd Layer
model1.add(Conv3D(32, padding='same', kernel_size=(3, 3, 3),activation='relu'))
model1.add(BatchNormalization())

# 4th Layer
model1.add(Conv3D(32, padding='same', kernel_size=(3, 3, 3),activation='relu'))
model1.add(BatchNormalization())

# 5th Layer
model1.add(Conv3D(64, padding='same', kernel_size=(3, 3, 3),activation='relu'))
model1.add(BatchNormalization())

# 6th Layer
model1.add(Conv3D(64, padding='same', kernel_size=(3, 3, 3),activation='relu'))
model1.add(BatchNormalization())

model1.add(MaxPooling3D(pool_size=(2, 2, 2)))

# 7th Layer
model1.add(Conv3D(128, padding='same', kernel_size=(3, 3, 3),activation='relu'))
model1.add(BatchNormalization())

# 8th Layer
model1.add(Conv3D(128, padding='same', kernel_size=(3, 3, 3),activation='relu'))
model1.add(BatchNormalization())

model1.add(MaxPooling3D(pool_size=(2, 2, 2)))

model1.add(Flatten())

# Adding fully connected layer --1
model1.add(Dense(64,kernel_initializer="normal",activation='relu')) 

model1.add(BatchNormalization())
model1.add(Dropout(0.25))

# Adding fully connected layer --2
model1.add(Dense(64,kernel_initializer="normal",activation='relu')) 

model1.add(BatchNormalization())
model1.add(Dropout(0.25))

# softmax layer
model1.add(Dense(5, kernel_initializer="normal",activation='softmax'))

# model summary
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_21 (Conv3D)           (None, 25, 160, 120, 16)  1312      
_________________________________________________________________
batch_normalization_20 (Batc (None, 25, 160, 120, 16)  64        
_________________________________________________________________
conv3d_22 (Conv3D)           (None, 25, 160, 120, 16)  6928      
_________________________________________________________________
batch_normalization_21 (Batc (None, 25, 160, 120, 16)  64        
_________________________________________________________________
max_pooling3d_11 (MaxPooling (None, 12, 80, 60, 16)    0         
_________________________________________________________________
conv3d_23 (Conv3D)           (None, 12, 80, 60, 32)    13856     
_________________________________________________________________
batch_normalization_22 (Batc (None, 12, 80, 60, 32)    128       
__________

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [14]:
# Model--1
optimiser = optimizers.SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=True) 
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_3 (Conv3D)            (None, 23, 158, 118, 32)  2624      
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 7, 52, 39, 32)     0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 7, 52, 39, 32)     0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 5, 50, 37, 64)     55360     
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 1, 16, 12, 64)     0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 12288)             0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 12288)             0         
__________

In [15]:
# Model--2
optimiser1 = optimizers.Adam()
model1.compile(optimizer=optimiser1, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_21 (Conv3D)           (None, 25, 160, 120, 16)  1312      
_________________________________________________________________
batch_normalization_20 (Batc (None, 25, 160, 120, 16)  64        
_________________________________________________________________
conv3d_22 (Conv3D)           (None, 25, 160, 120, 16)  6928      
_________________________________________________________________
batch_normalization_21 (Batc (None, 25, 160, 120, 16)  64        
_________________________________________________________________
max_pooling3d_11 (MaxPooling (None, 12, 80, 60, 16)    0         
_________________________________________________________________
conv3d_23 (Conv3D)           (None, 12, 80, 60, 32)    13856     
_________________________________________________________________
batch_normalization_22 (Batc (None, 12, 80, 60, 32)    128       
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [16]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [17]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [18]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [14]:
# Training for Model --1
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=2, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = Source path =  Project_data/train ; batch size = 15
Epoch 1/60
15


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


 - 147s - loss: 1.6371 - categorical_accuracy: 0.2252 - val_loss: 1.5864 - val_categorical_accuracy: 0.2381

Epoch 00001: saving model to model_init_2019-09-2114_38_34.621521/model-00001-1.63710-0.22519-1.58640-0.23810.h5
Epoch 2/60
 - 72s - loss: 1.5695 - categorical_accuracy: 0.2667 - val_loss: 1.4955 - val_categorical_accuracy: 0.2857

Epoch 00002: saving model to model_init_2019-09-2114_38_34.621521/model-00002-1.56946-0.26667-1.49547-0.28571.h5
Epoch 3/60
 - 75s - loss: 1.5347 - categorical_accuracy: 0.2785 - val_loss: 1.5262 - val_categorical_accuracy: 0.3429

Epoch 00003: saving model to model_init_2019-09-2114_38_34.621521/model-00003-1.53468-0.27852-1.52617-0.34286.h5
Epoch 4/60
 - 79s - loss: 1.4636 - categorical_accuracy: 0.3763 - val_loss: 1.5044 - val_categorical_accuracy: 0.3619

Epoch 00004: saving model to model_init_2019-09-2114_38_34.621521/model-00004-1.46357-0.37630-1.50445-0.36190.h5
Epoch 5/60
 - 73s - loss: 1.4470 - categorical_accuracy: 0.3585 - val_loss: 1.5110

 - 82s - loss: 0.8856 - categorical_accuracy: 0.6622 - val_loss: 1.4380 - val_categorical_accuracy: 0.2952

Epoch 00036: saving model to model_init_2019-09-2114_38_34.621521/model-00036-0.88556-0.66222-1.43796-0.29524.h5
Epoch 37/60
 - 74s - loss: 0.9006 - categorical_accuracy: 0.6326 - val_loss: 1.4189 - val_categorical_accuracy: 0.2952

Epoch 00037: saving model to model_init_2019-09-2114_38_34.621521/model-00037-0.90062-0.63259-1.41886-0.29524.h5
Epoch 38/60
 - 81s - loss: 0.8939 - categorical_accuracy: 0.6474 - val_loss: 1.3761 - val_categorical_accuracy: 0.3238

Epoch 00038: saving model to model_init_2019-09-2114_38_34.621521/model-00038-0.89395-0.64741-1.37606-0.32381.h5

Epoch 00038: ReduceLROnPlateau reducing learning rate to 9.999999019782991e-06.
Epoch 39/60
 - 74s - loss: 0.9172 - categorical_accuracy: 0.6533 - val_loss: 1.3883 - val_categorical_accuracy: 0.2952

Epoch 00039: saving model to model_init_2019-09-2114_38_34.621521/model-00039-0.91724-0.65333-1.38832-0.29524.h5

In [ ]:
# Training for Model --2
model1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=2, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)